This notebook aims at reproducing results from Campos et al., more concretely, on the PageBlocks dataset with a contamination rate of 0.05 - see all datasets with that contamination rate on https://www.dbs.ifi.lmu.de/research/outlier-evaluation/DAMI/semantic/PageBlocks/. Specifically, the unscaled variant without duplicates is considered.

Campos et al. report the values for different `n_neighbors` for each sampled dataset. In the unscaled variant without duplicates, the result for LOF is frequently reported with `n_neighbors=100` or, if this is not the case, for a slighlty different parameter. In the following, we look at ROC AUC, AP and ajusted AP. For all metrics, the results on all 10 data sets sampled from PageBlocks are reported. If the `n_neighbors` parameter is not set to 100, this is indicated in parenthesis.

- ROC AUC: Average of 0.926. Individual values are: 0.932, 0.924, 0.921 (99), 0.925, 0.937, 0.925 (99), 0.932 (97), 0.919 (99), 0.925, 0.920.

- Average Precision: Average of 0.5085. Individual values are: 0.514, 0.486, 0.498 (99), 0.496, 0.564, 0.520 (99), 0.535 (97), 0.486 (99), 0.510, 0.476.

- Adjusted Average Precision: Average of 0.483. Individual values are: 0.490, 0.459, 0.472 (99), 0.469, 0.541, 0.494 (99), 0.511 (97), 0.459 (99), 0.484, 0.449.

In [1]:
import sys
sys.path.append('../..')

%load_ext autoreload
%autoreload 2

In [2]:
# other imports
import pandas as pd

from oab.data.classification_dataset import ClassificationDataset
from oab.data.unsupervised import UnsupervisedAnomalyDataset
from oab.data.load_dataset import load_dataset
from oab.evaluation import EvaluationObject, ComparisonObject

from pyod.models.lof import LOF
from pyod.models.knn import KNN

In [3]:
pageblocks_cd = load_dataset('page-blocks', anomaly_dataset=False, preprocess_classification_dataset=False)

Credits: Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science. For more information, check https://archive.ics.uci.edu/ml/datasets/Page+Blocks+Classification.


In [4]:
pageblocks_cd.delete_duplicates()
pageblocks_cd.treat_missing_values()
pageblocks_ad = UnsupervisedAnomalyDataset(pageblocks_cd, normal_labels=[1], anomaly_labels=[2, 3, 4, 5])

In [5]:
# run experiment
eval_obj = EvaluationObject(algorithm_name="LOF")
for (x, y), sample_config in pageblocks_ad.sample_multiple(n=5139, contamination_rate=0.05, n_steps=10):
    algo = LOF(n_neighbors=100)
    algo.fit(x)
    pred = algo.decision_scores_
    eval_obj.add(ground_truth=y, prediction=pred, description=sample_config)

_ = eval_obj.evaluate()

Evaluation on dataset page-blocks with normal labels [1] and anomaly labels [2, 3, 4, 5].
Total of 10 datasets. Per dataset:
5139 instances, contamination_rate 0.05.
Mean 	 Std_dev 	 Metric
0.926 	 0.005 		 roc_auc
0.506 	 0.018 		 average_precision
0.480 	 0.019 		 adjusted_average_precision


We can see that for LOF and `n_neighbors=100`, the ROC AUC we computed was `0.926`, which is what we expect. For AP, we expected `0.5085` and calculated `0.506`, and for Adjusted AP, the values are `0.483` and `0.480` respectively.